# Import libraries

In [1]:
import pandas as pd
from scipy.stats import wilcoxon
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import utils
import config
import re
import os

In [2]:
# set figure details
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

# Metadata

In [17]:
metadata = pd.read_csv(os.path.join(config.CLEAN_DIR, "metadata.tsv"), sep="\t", index_col=0)
metadata["date_time"] = pd.to_datetime(metadata['Date_Collected'])
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,Vaginal,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,Vaginal,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,C-section with labor,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23


In [21]:
for subject in metadata["Patient_No"].unique():
    subject_df = metadata[metadata["Patient_No"] == subject].sort_values("date_time")
    display(subject_df)
    input()

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0010,Knights_Project_076_Pool3,magic.0010.S167.001.fa,PowerSoil,7251673,2016-12-10
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15
magic.0519,101,no,6/11/17,NaN,24,regular,NaN,NaN,NaN,NaN,...,Vaginal,6.09,1.0,0.0,magic.0519,Knights_Project_076_Pool3,magic.0519.S220.001.fa,PowerSoil,5725244,2017-06-11
magic.0859,101,no,9/10/17,1630,36,regular,NaN,NaN,NaN,formed,...,Vaginal,9.08,1.0,0.0,magic.0859,Knights_Project_068_Pool1,magic.0859.S197.001.fa,PowerSoil,9289438,2017-09-10
magic.1142,101,no,12/11/17,1330,48,regular,NaN,NaN,NaN,formed,...,Vaginal,12.11,0.0,0.0,magic.1142,Knights_Project_076_Pool1,magic.1142.S49.001.fa,PowerSoil,6666588,2017-12-11
magic.1388,101,no,2/14/18,800,NaN,Abx,NaN,during,5,formed,...,Vaginal,14.24,0.0,0.0,magic.1388,Knights_Project_068_Pool2,magic.1388.S398.001.fa,PowerSoil,8328134,2018-02-14
magic.1574,101,no,2/28/18,730,NaN,Abx,NaN,after,10,formed,...,Vaginal,14.70,0.0,0.0,magic.1574,Knights_Project_068_Pool2,magic.1574.S498.001.fa,PowerSoil,8190565,2018-02-28
magic.1781,101,no,4/9/18,NaN,60,regular,NaN,NaN,NaN,hard,...,Vaginal,16.02,0.0,0.0,magic.1781,Knights_Project_068_Pool4,magic.1781.S936.001.fa,PowerSoil,8728277,2018-04-09


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15
magic.0020,104,no,12/21/16,1957,1,regular,NaN,NaN,NaN,watery,...,Vaginal,0.23,1.0,1.0,magic.0020,Knights_Project_046,magic.0020.S23.001.fa,MagAttract,1105790,2016-12-21
magic.0018,104,no,12/28/16,900,2,regular,NaN,NaN,NaN,watery,...,Vaginal,0.46,1.0,1.0,magic.0018,Knights_Project_046,magic.0018.S106.001.fa,MagAttract,2227237,2016-12-28
magic.0048,104,no,1/4/17,1344,3,regular,NaN,NaN,NaN,watery,...,Vaginal,0.69,1.0,1.0,magic.0048,Knights_Project_046,magic.0048.S83.001.fa,MagAttract,1068893,2017-01-04
magic.0078,104,no,1/11/17,2148,4,regular,NaN,NaN,NaN,watery,...,Vaginal,0.92,1.0,1.0,magic.0078,Knights_Project_046,magic.0078.S72.001.fa,MagAttract,1040799,2017-01-11
magic.0231,104,no,3/14/17,1500,12,regular,NaN,NaN,NaN,watery,...,Vaginal,2.96,1.0,1.0,magic.0231,Knights_Project_046,magic.0231.S63.001.fa,MagAttract,1210079,2017-03-14
magic.0527,104,no,6/14/17,1516,24,regular,NaN,NaN,NaN,watery,...,Vaginal,5.99,1.0,1.0,magic.0527,Knights_Project_046,magic.0527.S68.001.fa,MagAttract,1664226,2017-06-14
magic.0894,104,no,9/19/17,1017,36,regular,NaN,NaN,NaN,formed,...,Vaginal,9.18,1.0,0.0,magic.0894,Knights_Project_055,X894.S241.001.fa,PowerSoil,2213238,2017-09-19
magic.1115,104,no,12/14/17,855,48,regular,NaN,NaN,NaN,formed,...,Vaginal,12.01,1.0,0.0,magic.1115,Knights_Project_055,X1115.S329.001.fa,PowerSoil,2790906,2017-12-14


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15
magic.0012,105,no,12/22/16,950,1,regular,NaN,NaN,NaN,soft,...,Vaginal,0.26,1.0,1.0,magic.0012,Knights_Project_076_Pool3,magic.0012.S177.001.fa,PowerSoil,8106772,2016-12-22
magic.0017,105,no,12/28/16,1450,2,regular,NaN,NaN,NaN,watery,...,Vaginal,0.46,1.0,1.0,magic.0017,Knights_Project_046,magic.0017.S94.001.fa,MagAttract,1578676,2016-12-28
magic.0044,105,no,1/4/17,2137,3,regular,NaN,NaN,NaN,watery,...,Vaginal,0.69,1.0,1.0,magic.0044,Knights_Project_076_Pool3,magic.0044.S147.001.fa,PowerSoil,7582355,2017-01-04
magic.0064,105,no,1/11/17,2125,4,regular,NaN,NaN,NaN,watery,...,Vaginal,0.92,1.0,1.0,magic.0064,Knights_Project_055,X64.S126.001.fa,PowerSoil,3123543,2017-01-11
magic.0228,105,no,3/15/17,1700,12,regular,NaN,NaN,NaN,watery,...,Vaginal,2.99,1.0,1.0,magic.0228,Knights_Project_055,X228.S136.001.fa,PowerSoil,2965203,2017-03-15
magic.0541,105,no,6/14/17,1330,24,regular,NaN,NaN,NaN,formed,...,Vaginal,5.99,1.0,1.0,magic.0541,Knights_Project_046,magic.0541.S45.001.fa,MagAttract,1268539,2017-06-14
magic.0880,105,no,9/15/17,800,36,regular,NaN,NaN,NaN,soft,...,Vaginal,9.05,0.0,1.0,magic.0880,Knights_Project_055,X880.S187.001.fa,PowerSoil,2377697,2017-09-15


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0024,107,yes,12/19/16,1705,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,0.0,magic.0024,Knights_Project_055,X24.S133.001.fa,PowerSoil,3691700,2016-12-19
magic.0025,107,yes,12/20/16,407,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.07,1.0,0.0,magic.0025,Knights_Project_076_Pool3,magic.0025.S135.001.fa,PowerSoil,7428214,2016-12-20
magic.0011,107,no,12/25/16,1731,1,regular,NaN,NaN,NaN,watery,...,Vaginal,0.23,1.0,0.0,magic.0011,Knights_Project_046,magic.0011.S46.001.fa,MagAttract,1305578,2016-12-25
magic.0050,107,no,1/1/17,1754,2,regular,NaN,NaN,NaN,watery,...,Vaginal,0.46,1.0,0.0,magic.0050,Knights_Project_046,magic.0050.S95.001.fa,MagAttract,1105272,2017-01-01
magic.0060,107,no,1/9/17,1430,3,regular,NaN,NaN,NaN,soft,...,Vaginal,0.72,1.0,0.0,magic.0060,Knights_Project_068_Pool1,magic.0060.S2.001.fa,PowerSoil,8633339,2017-01-09
magic.0080,107,no,1/14/17,2130,4,regular,NaN,NaN,NaN,soft,...,Vaginal,0.89,1.0,0.0,magic.0080,Knights_Project_055,X80.S173.001.fa,PowerSoil,2336036,2017-01-14
magic.0246,107,no,3/14/17,1145,12,regular,NaN,NaN,NaN,watery,...,Vaginal,2.83,1.0,0.0,magic.0246,Knights_Project_046,magic.0246.S28.001.fa,MagAttract,941861,2017-03-14
magic.0538,107,no,6/17/17,1600,24,regular,NaN,NaN,NaN,soft,...,Vaginal,5.95,1.0,0.0,magic.0538,Knights_Project_046,magic.0538.S116.001.fa,MagAttract,1453847,2017-06-17
magic.0893,107,no,9/18/17,1330,36,regular,NaN,NaN,NaN,formed,...,Vaginal,9.01,1.0,0.0,magic.0893,Knights_Project_055,X893.S229.001.fa,PowerSoil,2314616,2017-09-18


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0013,102,no,12/19/16,2142,1,regular,NaN,NaN,NaN,watery,...,Vaginal,0.23,1.0,0.0,magic.0013,Knights_Project_046,magic.0013.S58.001.fa,MagAttract,1263330,2016-12-19
magic.0023,102,no,12/26/16,1855,2,regular,NaN,NaN,NaN,soft,...,Vaginal,0.46,1.0,0.0,magic.0023,Knights_Project_068_Pool1,magic.0023.S13.001.fa,PowerSoil,13305674,2016-12-26
magic.0049,102,no,1/2/17,1050,3,regular,NaN,NaN,NaN,soft,...,Vaginal,0.69,1.0,0.0,magic.0049,Knights_Project_068_Pool1,magic.0049.S83.001.fa,PowerSoil,10981787,2017-01-02
magic.0051,102,no,1/11/17,2130,4,regular,NaN,NaN,NaN,watery,...,Vaginal,0.99,1.0,0.0,magic.0051,Knights_Project_055,X51.S110.001.fa,PowerSoil,3144620,2017-01-11
magic.0230,102,no,3/12/17,1145,12,regular,NaN,NaN,NaN,soft,...,Vaginal,2.96,1.0,0.0,magic.0230,Knights_Project_055,X230.S144.001.fa,PowerSoil,2405023,2017-03-12
magic.0526,102,no,6/8/17,1200,24,Abx_regular,NaN,after,2,soft,...,Vaginal,5.86,1.0,0.0,magic.0526,Knights_Project_046,magic.0526.S56.001.fa,MagAttract,1325372,2017-06-08
magic.0532,102,no,6/16/17,732,24,Abx_regular,NaN,after,10,soft,...,Vaginal,6.12,1.0,0.0,magic.0532,Knights_Project_046,magic.0532.S92.001.fa,MagAttract,1379597,2017-06-16
magic.0627,102,no,7/10/17,1925,na,Abx,yes,after,30,hard,...,Vaginal,6.91,1.0,0.0,magic.0627,Knights_Project_068_Pool1,magic.0627.S61.001.fa,PowerSoil,8652008,2017-07-10
magic.0848,102,no,9/12/17,1200,36,regular,NaN,NaN,NaN,hard,...,Vaginal,9.01,1.0,0.0,magic.0848,Knights_Project_068_Pool1,magic.0848.S231.001.fa,PowerSoil,10962430,2017-09-12


...

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0028,108,yes,12/20/16,1620,0,regular,NaN,NaN,NaN,NaN,...,C-section with labor,0.03,1.0,1.0,magic.0028,Knights_Project_046,magic.0028.S47.001.fa,MagAttract,501831,2016-12-20


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0030,119,yes,1/11/17,900,0,regular,NaN,NaN,NaN,NaN,...,C-section without labor,0.07,1.0,0.0,magic.0030,Knights_Project_046,magic.0030.S59.001.fa,MagAttract,1180180,2017-01-11
magic.0031,119,yes,1/12/17,145,0,regular,NaN,NaN,NaN,NaN,...,C-section without labor,0.10,1.0,0.0,magic.0031,Knights_Project_076_Pool3,magic.0031.S156.001.fa,PowerSoil,5646899,2017-01-12
magic.0038,119,no,1/16/17,424,1,regular,NaN,NaN,NaN,soft,...,C-section without labor,0.23,1.0,0.0,magic.0038,Knights_Project_068_Pool1,magic.0038.S48.001.fa,PowerSoil,10055070,2017-01-16
magic.0074,119,no,1/23/17,1628,2,regular,NaN,NaN,NaN,soft,...,C-section without labor,0.46,1.0,0.0,magic.0074,Knights_Project_076_Pool3,magic.0074.S190.001.fa,PowerSoil,6081525,2017-01-23
magic.0108,119,no,1/30/17,1700,3,regular,NaN,NaN,NaN,soft,...,C-section without labor,0.69,1.0,0.0,magic.0108,Knights_Project_068_Pool1,magic.0108.S3.001.fa,PowerSoil,9507614,2017-01-30
magic.0083,119,no,2/6/17,1400,4,regular,NaN,NaN,NaN,soft,...,C-section without labor,0.92,1.0,0.0,magic.0083,Knights_Project_076_Pool3,magic.0083.S148.001.fa,PowerSoil,9038009,2017-02-06
magic.0633,119,no,7/10/17,1310,24,regular,NaN,NaN,NaN,soft,...,C-section without labor,5.99,1.0,0.0,magic.0633,Knights_Project_068_Pool1,magic.0633.S153.001.fa,PowerSoil,25335,2017-07-10
magic.1257,119,no,1/9/18,1520,48,regular,NaN,NaN,NaN,formed,...,C-section without labor,12.01,1.0,0.0,magic.1257,Knights_Project_068_Pool2,magic.1257.S438.001.fa,PowerSoil,7302133,2018-01-09
magic.2372,119,no,7/10/18,1830,72,regular,NaN,NaN,NaN,soft,...,C-section without labor,17.99,0.0,0.0,magic.2372,Knights_Project_068_Pool3,magic.2372.S748.001.fa,PowerSoil,11317762,2018-07-10


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0033,122,yes,1/13/17,1511,0,regular,NaN,NaN,NaN,NaN,...,Vaginal,0.03,1.0,0.0,magic.0033,Knights_Project_076_Pool3,magic.0033.S168.001.fa,PowerSoil,32546587,2017-01-13
magic.0112,122,no,1/26/17,1600,2,regular,NaN,NaN,NaN,watery,...,Vaginal,0.46,1.0,0.0,magic.0112,Knights_Project_046,magic.0112.S49.001.fa,MagAttract,885824,2017-01-26
magic.0085,122,no,2/2/17,1015,3,regular,NaN,NaN,NaN,watery,...,Vaginal,0.69,1.0,0.0,magic.0085,Knights_Project_068_Pool1,magic.0085.S37.001.fa,PowerSoil,10313171,2017-02-02
magic.0133,122,no,2/9/17,1230,4,regular,NaN,NaN,NaN,watery,...,Vaginal,0.92,1.0,0.0,magic.0133,Knights_Project_068_Pool1,magic.0133.S85.001.fa,PowerSoil,6071972,2017-02-09
magic.0643,122,no,7/12/17,1246,24,regular,NaN,NaN,NaN,watery,...,Vaginal,5.95,1.0,0.0,magic.0643,Knights_Project_068_Pool1,magic.0643.S189.001.fa,PowerSoil,11316177,2017-07-12
magic.1295,122,no,1/12/18,1530,48,regular,NaN,NaN,NaN,formed,...,Vaginal,12.01,1.0,0.0,magic.1295,Knights_Project_068_Pool2,magic.1295.S405.001.fa,PowerSoil,9369194,2018-01-12
magic.1805,122,no,4/12/18,1030,60,regular,NaN,NaN,NaN,formed,...,Vaginal,14.97,1.0,0.0,magic.1805,Knights_Project_068_Pool3,magic.1805.S575.001.fa,PowerSoil,11337963,2018-04-12
magic.2365,122,no,7/12/18,1000,72,regular,NaN,NaN,NaN,soft,...,Vaginal,17.96,1.0,0.0,magic.2365,Knights_Project_076_Pool1,magic.2365.S330.001.fa,PowerSoil,6283977,2018-07-12
magic.2759,122,no,10/12/18,1200,84,regular,NaN,NaN,NaN,soft,...,Vaginal,20.99,1.0,0.0,magic.2759,Knights_Project_068_Pool4,magic.2759.S821.001.fa,PowerSoil,8412440,2018-10-12


,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,expo_cat2,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0035,117,yes,1/6/17,2100,0,regular,NaN,NaN,NaN,NaN,...,C-section with labor,0.03,1.0,0.0,magic.0035,Knights_Project_076_Pool3,magic.0035.S189.001.fa,PowerSoil,6417743,2017-01-06
magic.0034,117,yes,1/7/17,930,0,regular,NaN,NaN,NaN,NaN,...,C-section with labor,0.07,1.0,0.0,magic.0034,Knights_Project_076_Pool3,magic.0034.S178.001.fa,PowerSoil,29213402,2017-01-07
magic.0040,117,no,1/13/17,1100,1,regular,NaN,NaN,NaN,soft,...,C-section with labor,0.26,1.0,0.0,magic.0040,Knights_Project_046,magic.0040.S71.001.fa,MagAttract,1267753,2017-01-13
magic.0111,117,no,1/19/17,1930,2,regular,NaN,NaN,NaN,soft,...,C-section with labor,0.46,1.0,0.0,magic.0111,Knights_Project_068_Pool1,magic.0111.S15.001.fa,PowerSoil,8335658,2017-01-19
magic.0136,117,no,1/26/17,900,3,regular,NaN,NaN,NaN,soft,...,C-section with labor,0.69,1.0,0.0,magic.0136,Knights_Project_076_Pool3,magic.0136.S192.001.fa,PowerSoil,5769735,2017-01-26
magic.0104,117,no,2/2/17,2300,4,regular,NaN,NaN,NaN,soft,...,C-section with labor,0.92,1.0,0.0,magic.0104,Knights_Project_076_Pool3,magic.0104.S191.001.fa,PowerSoil,7405592,2017-02-02
magic.0638,117,no,7/5/17,1800,24,regular,NaN,NaN,NaN,soft,...,C-section with labor,5.95,1.0,0.0,magic.0638,Knights_Project_068_Pool1,magic.0638.S177.001.fa,PowerSoil,12836628,2017-07-05
magic.1045,117,no,10/9/17,815,36,regular,NaN,NaN,NaN,soft,...,C-section with labor,9.11,1.0,0.0,magic.1045,Knights_Project_068_Pool2,magic.1045.S300.001.fa,PowerSoil,31638517,2017-10-09
magic.1043,117,no,10/29/17,1645,NaN,Abx,NaN,during,5,formed,...,C-section with labor,9.77,NaN,NaN,magic.1043,Knights_Project_068_Pool2,magic.1043.S368.001.fa,PowerSoil,15215656,2017-10-29


KeyboardInterrupt: Interrupted by user

In [4]:
richness = pd.read_csv(os.path.join(config.CLEAN_DIR, "rarified_gene_richness.tsv"), sep="\t", index_col=0)
richness

,rarified_gene_richness
magic.0001,64331.0
magic.0004,69105.0
magic.0006,21372.0
magic.0007,89717.0
magic.0008,61474.0
...,...
magic.6595,185364.0
magic.6597,176493.0
magic.6598,142285.0
magic.6599,155385.0


In [5]:
assert(all(metadata.index == richness.index))
metadata["gene_richness"] = richness
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15,21372.0
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15,89717.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15,61474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22,185364.0
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06,176493.0
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23,142285.0


In [6]:
metadata["abx_day"].value_counts()

10         232
2          193
5          186
30         135
0           83
na          26
1           16
6            6
7            6
3            5
4            3
8            2
13           2
14           2
9            2
20           2
31           1
11           1
32           1
5 or 10      1
2 & 5        1
Na           1
Name: abx_day, dtype: int64

In [7]:
metadata["Timeline_Weeks"].value_counts()

4     307
1     301
24    298
12    278
48    274
96    220
3     184
2     169
0     162
72    149
36    112
60     73
na     50
84     41
Name: Timeline_Weeks, dtype: int64

# subject gene richness z score for subject age vs abx state

In [8]:
from scipy.stats import zscore

zscores_list = []

for week in metadata["Timeline_Weeks"].unique():
    if week == "na": continue
    gene_richness = metadata[metadata["Timeline_Weeks"] == week]["gene_richness"]
    zscores = zscore(gene_richness)
    zscores_list.append(pd.DataFrame(zscores))

In [9]:
zscore_df = pd.concat(zscores_list, axis=0).iloc[:, [0]]
zscore_df.columns = ["zscore"]
zscore_df

,zscore
magic.0001,0.500181
magic.0006,-0.839210
magic.0007,1.291674
magic.0008,0.411104
magic.0010,0.432773
...,...
magic.6595,1.304466
magic.6597,1.069306
magic.6598,0.162491
magic.6599,0.509757


In [10]:
zscore_df

,zscore
magic.0001,0.500181
magic.0006,-0.839210
magic.0007,1.291674
magic.0008,0.411104
magic.0010,0.432773
...,...
magic.6595,1.304466
magic.6597,1.069306
magic.6598,0.162491
magic.6599,0.509757


In [11]:
metadata[metadata["Abx_regular"] == "Abx"]["gene_richness"].mean()

100589.5172413793

In [12]:
for subject in metadata["Patient_No"].unique():
    display(metadata[metadata["Patient_No"] == subject])
    input()
    

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0010,101,no,12/10/16,2230,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0010,Knights_Project_076_Pool3,magic.0010.S167.001.fa,PowerSoil,7251673,2016-12-10,62169.0
magic.0016,101,no,NaN,NaN,2,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0016,Knights_Project_046,magic.0016.S82.001.fa,MagAttract,979090,NaT,65804.0
magic.0022,101,no,NaN,NaN,3,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0022,Knights_Project_076_Pool3,magic.0022.S198.001.fa,PowerSoil,23427901,NaT,83135.0
magic.0042,101,no,NaN,NaN,4,regular,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,magic.0042,Knights_Project_068_Pool1,magic.0042.S60.001.fa,PowerSoil,8397705,NaT,95631.0
magic.0519,101,no,6/11/17,NaN,24,regular,NaN,NaN,NaN,NaN,...,6.09,1.0,0.0,magic.0519,Knights_Project_076_Pool3,magic.0519.S220.001.fa,PowerSoil,5725244,2017-06-11,88680.0
magic.0859,101,no,9/10/17,1630,36,regular,NaN,NaN,NaN,formed,...,9.08,1.0,0.0,magic.0859,Knights_Project_068_Pool1,magic.0859.S197.001.fa,PowerSoil,9289438,2017-09-10,111584.0
magic.1142,101,no,12/11/17,1330,48,regular,NaN,NaN,NaN,formed,...,12.11,0.0,0.0,magic.1142,Knights_Project_076_Pool1,magic.1142.S49.001.fa,PowerSoil,6666588,2017-12-11,104975.0


KeyboardInterrupt: Interrupted by user

In [13]:
subjects = metadata["Patient_No"].unique()
subjects

array([101, 104, 105, 107, 102, 106, 109, 108, 119, 122, 117, 115, 120,
       111, 116, 125, 124, 126, 127, 136, 131, 137, 135, 142, 133, 140,
       144, 143, 141, 123, 149, 148, 151, 150, 147, 155, 153, 158, 162,
       160, 145, 161, 167, 166, 168, 169, 165, 170, 157, 164, 172, 174,
       173, 181, 185, 180, 175, 187, 183, 177, 184, 186, 188, 189, 193,
       198, 195, 132, 192, 190, 191, 196, 197, 199, 138, 202, 204, 203,
       208, 201, 205, 209, 211, 210, 217, 219, 218, 222, 224, 220, 221,
       223, 225, 213, 227, 233, 229, 231, 226, 235, 228, 247, 237, 238,
       239, 245, 243, 241, 249, 242, 253, 250, 251, 263, 256, 254, 258,
       257, 266, 260, 261, 267, 264, 248, 265, 268, 259, 270, 252, 271,
       269, 275, 276, 272, 280, 277, 282, 281, 234, 273, 283, 285, 289,
       286, 287, 290, 293, 298, 299, 297, 300, 302, 301, 295, 308, 307,
       312, 304, 305, 274, 313, 318, 296, 306, 315, 322, 319, 324, 333,
       328, 321, 329, 330, 332, 323, 341, 335, 327, 340, 337, 33

In [14]:
metadata

,Patient_No,Hospital,Date_Collected,Time_Collected,Timeline_Weeks,Abx_regular,Abx_anomoly,Abx_timeline.prior..during..after.,abx_day,Consistency,...,calc_age_mo,currentfeed_bf,currentfeed_f,currentfeed_o,Sequencing_Project,Filename,Extraction,depth,date_time,gene_richness
Sample_ID,,,,,,,,,,,,,,,,,,,,,
magic.0001,101,no,12/10/16,1430,0,regular,NaN,NaN,NaN,NaN,...,0.07,1.0,0.0,magic.0001,Knights_Project_046,magic.0001.S22.001.fa,MagAttract,1544781,2016-12-10,64331.0
magic.0004,101,no,12/15/16,NaN,1,regular,NaN,NaN,NaN,NaN,...,0.23,1.0,0.0,magic.0004,Knights_Project_046,magic.0004.S34.001.fa,MagAttract,1167038,2016-12-15,69105.0
magic.0006,104,yes,12/15/16,2335,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0006,Knights_Project_076_Pool3,magic.0006.S145.001.fa,PowerSoil,1720636,2016-12-15,21372.0
magic.0007,105,yes,12/15/16,2115,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0007,Knights_Project_076_Pool3,magic.0007.S155.001.fa,PowerSoil,6825017,2016-12-15,89717.0
magic.0008,105,yes,12/15/16,1525,0,regular,NaN,NaN,NaN,NaN,...,0.03,1.0,1.0,magic.0008,Knights_Project_055,X8.S125.001.fa,PowerSoil,2928044,2016-12-15,61474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
magic.6595,588,NaN,8/22/21,1702,96,regular,NaN,NaN,NaN,NaN,...,24.11,NaN,NaN,magic.6595,Knights_Project_077_Pool3,magic.6595.S253.001.fa,PowerSoilPro,1562968,2021-08-22,185364.0
magic.6597,595,NaN,9/6/21,NaN,96,regular,NaN,NaN,NaN,formed,...,24.11,NaN,NaN,magic.6597,Knights_Project_077_Pool3,magic.6597.S265.001.fa,PowerSoilPro,2113880,2021-09-06,176493.0
magic.6598,585,NaN,8/23/21,725,96,regular,NaN,NaN,NaN,formed,...,24.34,NaN,NaN,magic.6598,Knights_Project_077_Pool3,magic.6598.S277.001.fa,PowerSoilPro,1650388,2021-08-23,142285.0


In [15]:
ptr = pd.read_csv(os.path.join(config.CLEAN_DIR, "ptr.csv"), index_col=0)
ptr

,magic.0001,magic.0004,magic.0006,magic.0007,magic.0008,magic.0010,magic.0011,magic.0012,magic.0013,magic.0014,...,magic.6590,magic.6591,magic.6592,magic.6593,magic.6594,magic.6595,magic.6597,magic.6598,magic.6599,magic.6600
gtdb_taxonomy,,,,,,,,,,,,,,,,,,,,,
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Streptococcaceae;g__Streptococcus;s__Streptococcus thermophilus,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.890374,0.000000,0.000000,0.000000,0.000000,0.000000,0.398078,0.000000,0.443034
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.181598,0.000000,0.000000,0.000000,0.153008,0.164560,0.000000,0.181179,0.000000,0.364100
d__Bacteria;p__Verrucomicrobia;c__Verrucomicrobiae;o__Verrucomicrobiales;f__Akkermansiaceae;g__Akkermansia;s__Akkermansia muciniphila,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.531152,0.000000,0.000000,0.000000,0.000000,0.385291,0.339425
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Ruminiclostridium_E;s__Ruminiclostridium_E siraeum,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.366896,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.0,0.0,0.0,0.0,0,0.0,0.439616,0.0,0.0,0.0,...,0.000000,0.648231,0.639207,0.000000,0.279416,0.547436,0.706284,0.624992,0.793504,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d__Bacteria;p__Firmicutes;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Blautia;s__,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Actinomycetaceae;g__Actinomyces_B;s__,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
d__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Betaproteobacteriales;f__Burkholderiaceae;g__Diaphorobacter;s__Diaphorobacter ebreus,0.0,0.0,0.0,0.0,0,0.0,0.000000,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [16]:
from scipy.stats import ranksums

for week in metadata["Timeline_Weeks"].unique():
    if str(week) in ["nan", "na"]: continue
    no_abx = metadata[(metadata["expo_cat1"] == "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
    abx = metadata[(metadata["expo_cat1"] != "No antibiotics") & (metadata["Timeline_Weeks"] == week)].index
    ptr_noabx = ptr[no_abx]
    ptr_abx = ptr[abx]
    
    wilc_info = []
    
    for bug in ptr.index:
        if len(ptr_noabx.loc[bug].shape) == 2: continue
        info = ranksums(ptr_noabx.loc[bug], ptr_abx.loc[bug])
        stat, p = info.statistic, info.pvalue
        wilc_info.append({"p" : p, "stat" : stat, "bug" : bug})
        
    wilc_info_df = pd.DataFrame(wilc_info)
    wilc_info_df = wilc_info_df.set_index("bug")
    sorted_ = wilc_info_df.sort_values("p")
    print("week", week)
    display(sorted_[sorted_["p"] < 0.05])

week 0


,p,stat
bug,,


week 1


,p,stat
bug,,
d__Bacteria;p__Actinobacteria;c__Actinobacteria;o__Actinomycetales;f__Bifidobacteriaceae;g__Bifidobacterium;s__Bifidobacterium longum,0.000035,4.136309
d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;f__Enterococcaceae;g__Enterococcus;s__Enterococcus faecalis,0.000160,-3.774672


week 2


...

,p,stat
bug,,


week 84


,p,stat
bug,,


week 96


,p,stat
bug,,
d__Bacteria;p__Firmicutes;c__Clostridia;o__Oscillospirales;f__Oscillospiraceae;g__Intestinimonas;s__Intestinimonas butyriciproducens,0.000483,3.490159
